In [ ]:
%load_ext autoreload
%autoreload 2

from __future__ import annotations

import os

os.environ['USE_PYGEOS'] = '0'  # remove geopandas warnings

import holoviews as hv
import numpy as np
import numpy_indexed as npi
import pandas as pd
import xarray as xr

hv.extension("bokeh")

## Subsetting an unstructured mesh using a BBox 

In unstructured meshes the `longitude` and `latitude` are data variables instead of dimensions.
This means that filtering/subsetting a mesh using a Bounding Box (BBox) is somewhat more challenging.
Nevertheless it is still possible to do it using `numpy` and `xarray`.

This notebook describes how to achieve this.

For convenience, we are going to use an ADCIRC netcdf file which has been converted to the "Thalassa schema".

We will also need an additional library which the following cell will take care of importing.

In [ ]:
try:
    import numpy_indexed
except ImportError:
    %pip install numpy_indexed
    

### Sample file

As an example let's download a netcdf file containing the output of an ADCIRC model.

The file will be saved at `/tmp/fort.63.nc`:

In [ ]:
%%bash
pushd /tmp
wget --quiet https://github.com/ec-jrc/Thalassa/files/10867068/fort.63.zip 
unzip -o fort.63.zip
popd

Let's load the file and convert it to the thalassa schema.

In [ ]:
from thalassa import api

filename = "/tmp/fort.63.nc"
ds = api.open_dataset(filename)
ds

As we can see the model has 3070 nodes and 5780 trifaces (i.e. triangles)

Note: The reason why the thalassa schema defines `triface` is because `face` might be referring to quad elements, too (in Schism models). `triface` always refers to triangles.

Now let's define the function that does the filtering by BBox

In [ ]:
def filter_thalassa_by_bbox(
    ds: xr.Dataset,
    lon_min: float,
    lon_max: float,
    lat_min: float,
    lat_max: float
) -> xr.Dataset:
    indices_of_nodes_in_bbox = np.where(
        True 
        & (ds.lat >= lat_min)
        & (ds.lat <= lat_max) 
        & (ds.lon >= lon_min) 
        & (ds.lon <= lon_max)
    )[0]
    indices_of_triface_nodes_in_bbox = np.where(
        ds.triface_nodes.isin(indices_of_nodes_in_bbox).all(axis=1)
    )[0]
    ds = ds.isel(node=indices_of_nodes_in_bbox, triface=indices_of_triface_nodes_in_bbox)
    remapped_nodes = np.arange(len(indices_of_nodes_in_bbox))
    remapped_triface_nodes = np.c_[
        npi.remap(ds.triface_nodes[:,0], indices_of_nodes_in_bbox, remapped_nodes),
        npi.remap(ds.triface_nodes[:,1], indices_of_nodes_in_bbox, remapped_nodes),
        npi.remap(ds.triface_nodes[:,2], indices_of_nodes_in_bbox, remapped_nodes),
    ]
    ds["triface_nodes"] = (("triface", "three"), remapped_triface_nodes)
    return ds

Let's see it in action!

The original Dataset depicts the Hampton port/Bay. 
The BBox we used centers in the inner harbor area. 

`fds` is the filtered Dataset and as we can see it only contains 798 nodes and 1325 trifaces

In [ ]:
fds = filter_thalassa_by_bbox(
    ds=ds,
    lon_min=-72.6,
    lon_max=-72.4,
    lat_min=40.8,
    lat_max=41.0,
)

ds.dims
fds.dims

Let's use `thalassa` to visualize the difference:

In [ ]:
trimesh = api.create_trimesh(
    ds=ds,
    variable="zeta",
    timestamp="max",
    layer=None,
)

In [ ]:
dmap = api.get_tiles() * api.get_raster(trimesh=trimesh) #* api.get_wireframe(trimesh=trimesh)
dmap.opts(width=600, title="Full dataset")

In [ ]:
ftrimesh = api.create_trimesh(
    ds=fds,
    variable="zeta",
    timestamp="max",
    layer=None,
)
dmap = api.get_tiles() * api.get_raster(trimesh=ftrimesh) * api.get_wireframe(trimesh=ftrimesh)
dmap.opts(width=600, title="Filtered Dataset")

### Further development

It should not be too difficult to extend/modify the function so that: 

1. it supports different Dataset schemas (e.g. vanilla ADCIRC, vanilla SCHISM) but this is left as an exercise to the reader.
2. it filters using a vector object (e.g. a shapefile) instead of a simple BBox.